In [ ]:
%pip install -qU kaggle

In [ ]:
from google.colab import files
files.upload()   # choose kaggle.json you downloaded from your Kaggle account page
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle competitions download -c california-house-prices -p /content/data

  0% 0.00/29.5M [00:00<?, ?B/s]
100% 29.5M/29.5M [00:00<00:00, 1.28GB/s]


In [ ]:
!unzip -q /content/data/*.zip -d /content/data

In [ ]:
!pip -q install -U autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.9/454.9 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 14.6 MB/s eta 0:0

In [ ]:
import pandas as pd
import numpy as np
import argparse
import os
import random
from autogluon.tabular import TabularPredictor
from autogluon.multimodal import MultiModalPredictor
import torch as th

def get_parser():
    parser = argparse.ArgumentParser(
        description='The Basic Example of AutoGluon for House Price Prediction.')
    parser.add_argument('--mode',
                        choices=['stack5',
                                 'weighted',
                                 'single',
                                 'single_bag5'],
                        default='weighted',
                        help='"stack5" means 5-fold stacking. "weighted" means weighted ensemble.'
                             ' "single" means use a single model.'
                             ' "single_bag5" means 5-fold bagging via the AutoMM model.')
    parser.add_argument('--automm-mode', choices=['ft-transformer', 'mlp'],
                        default='ft-transformer', help='Fusion model in AutoMM.')
    parser.add_argument('--text-backbone', default='google/electra-small-discriminator')
    parser.add_argument('--cat-as-text', default=False)
    parser.add_argument('--data_path', type=str, default='california-house-prices')
    parser.add_argument('--seed', type=int, default=123)
    parser.add_argument('--exp_path', default=None)
    parser.add_argument('--with_tax_values', default=1, type=int)
    return parser

def get_automm_hyperparameters(mode, text_backbone, cat_as_text):
    if mode == "ft-transformer":
        hparams = {"model.names": ["ft_transformer",
                                   "hf_text",
                                   "fusion_transformer"],
                   "model.hf_text.checkpoint_name": text_backbone,
                   "data.categorical.convert_to_text": cat_as_text}
    elif mode == "mlp":
        hparams = {"model.names": ["categorical_mlp",
                                   "numerical_mlp",
                                   "hf_text",
                                   "fusion_mlp"],
                   "model.hf_text.checkpoint_name": text_backbone,
                   "data.categorical.convert_to_text": cat_as_text}
    else:
        raise NotImplementedError(f"mode={mode} is not supported!")
    return hparams


def preprocess(df, with_tax_values=True, log_scale_lot=True,
               log_scale_listed_price=True, has_label=True):
    new_df = df.copy()
    new_df.drop('Id', axis=1, inplace=True)
    new_df['Elementary School'] = new_df['Elementary School'].apply(lambda ele: str(ele)[:-len(' Elementary School')] if str(ele).endswith('Elementary School') else ele)
    epsilon = 1e-9  # Small value to avoid log(0)
    if log_scale_lot:
        new_df['Lot'] = np.log(new_df['Lot'] + epsilon + 1)
    if log_scale_listed_price:
        log_listed_price = np.log(new_df['Listed Price'] + epsilon).clip(0, None)
        new_df['Listed Price'] = log_listed_price
    if with_tax_values:
        new_df['Tax assessed value'] = np.log(new_df['Tax assessed value'] + epsilon + 1)
        new_df['Annual tax amount'] = np.log(new_df['Annual tax amount'] + epsilon + 1)
    else:
        new_df.drop('Tax assessed value', axis=1, inplace=True)
        new_df.drop('Annual tax amount', axis=1, inplace=True)
    if has_label:
        new_df['Sold Price'] = np.log(new_df['Sold Price'] + epsilon)
    return new_df


def set_seed(seed):
    import torch as th
    th.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    return seed


def train(args):
    seed = 123
    seed = set_seed(seed)
    args['seed'] = seed

    train_df = pd.read_csv(os.path.join(args['data_path'], 'train.csv'))
    test_df = pd.read_csv(os.path.join(args['data_path'], 'test.csv'))
    # For the purpose of generating submission file
    submission_df = pd.read_csv(os.path.join(args['data_path'], 'sample_submission.csv'))
    train_df = preprocess(train_df,
                          with_tax_values=args['with_tax_values'], has_label=True)
    test_df = preprocess(test_df,
                         with_tax_values=args['with_tax_values'], has_label=False)
    label_column = 'Sold Price'
    eval_metric = 'r2'

    automm_hyperparameters = get_automm_hyperparameters(args['automm_mode'], args['text_backbone'], args['cat_as_text'])
    automm_hyperparameters.update({
        "optim.max_epochs": 3,   # <- run at most 5 epochs
        "optim.patience": 1,     # <- stop early if no val improvement
    })
    tabular_hyperparameters = {
        'GBM': [
            {},
            {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
        ],
        'CAT': {},
        'AG_AUTOMM': automm_hyperparameters,
    }
    if args['mode'] == 'single':
        predictor = MultiModalPredictor(eval_metric=eval_metric, label=label_column, path=args['exp_path'])
        predictor.fit(train_df, hyperparameters=automm_hyperparameters, seed=args['seed'])
    elif args['mode'] == 'weighted' or args['mode'] == 'stack5' or args['mode'] == 'single_bag5' or args['mode'] == 'single_bag4':
        predictor = TabularPredictor(eval_metric=eval_metric, label=label_column, path=args['exp_path'])

        if args['mode'] == 'single_bag5':
            tabular_hyperparameters = {
                'AG_AUTOMM': automm_hyperparameters,
            }
            num_bag_folds, num_stack_levels = 5, 0
        elif args['mode'] == 'weighted':
            num_bag_folds, num_stack_levels = None, None
        elif args['mode'] == 'stack5':
            num_bag_folds, num_stack_levels = 5, 1
        else:
            raise NotImplementedError
        predictor.fit(train_df,
                      hyperparameters=tabular_hyperparameters,
                      num_bag_folds=num_bag_folds,
                      num_stack_levels=num_stack_levels)
        leaderboard = predictor.leaderboard()
        leaderboard.to_csv(os.path.join(args['exp_path'], 'leaderboard.csv'))
    else:
        raise NotImplementedError
    return predictor, test_df, submission_df

def predict(predictor, test_df, submission_df):
    predictions = np.exp(predictor.predict(test_df))
    submission_df['Sold Price'] = predictions
    submission_df.to_csv(os.path.join(args['exp_path'], 'submission.csv'), index=None)

In [ ]:
rm -r /content/automm_**

In [ ]:
args={'exp_path':None,
      'mode':'single',
      'automm_mode':'mlp',
      'cat_as_text':False,
      'text_backbone':'google/electra-small-discriminator',
      'with_tax_values':1,
      'data_path':'/content/data',
      'seed':123
      }

args['exp_path'] = f'automm_kaggle_house_{args['mode']}_{args['automm_mode']}_cat_to_text{args['cat_as_text']}_{args['text_backbone']}'

predictor, test_df, submission_df = train(args)

=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.7.1+cu126
CUDA Version:       12.6
GPU Count:          1
Memory Avail:       6.78 GB / 12.67 GB (53.5%)
Disk Space Avail:   189.16 GB / 235.68 GB (80.3%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/automm_kaggle_house_single_mlp_cat_to_textFalse_google/electra-small-discriminator
    ```

INFO: Seed set to 123
GPU Count: 1
GPU Count to be Used: 1

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEV

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 175: 'val_r2' reached 0.77891 (best 0.77891), saving model to '/content/automm_kaggle_house_single_mlp_cat_to_textFalse_google/electra-small-discriminator/epoch=0-step=175.ckpt' as top 3


...

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/automm_kaggle_house_single_mlp_cat_to_textFalse_google/electra-small-discriminator")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




In [ ]:
predict(predictor, test_df, submission_df)

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:

# args={'exp_path':None,
#       'mode':'single',
#       'automm_mode':'ft-transformer',
#       'cat_as_text':False,
#       'text_backbone':'google/electra-small-discriminator',
#       'with_tax_values':1,
#       'data_path':'/content/data',
#       'seed':123
#       }

# args['exp_path'] = f'automm_kaggle_house_{args['mode']}_{args['automm_mode']}_cat_to_text{args['cat_as_text']}_{args['text_backbone']}'

# train(args)